In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [2]:
PROBLEM_SIZE = 5        #number of elements in the set
NUM_SETS = 10       #number of sets
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)   # creates tuple of num_sets np.array where each contains problem_size elements with 30% prob of being true
State = namedtuple('State', ['taken', 'not_taken']) 
# each state is defined by the taken sets and the not-taken sets, can be accessed similarly to a dict
# note that it saves the index of the subset taken and not the actual subset

In [3]:
# chekes if I actually took all the sets with an or executed between all taken sets for that particular state
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))
# starts with an array of False (the inial)
# [SETS[i] for i in state.taken] creates a list of lists (the taken subsets)
# applies `or` to each of element of the first subset and to the initial
# then it applies the or to each elements of the second subset and the new initial
# goes on like this, if everything inside initial is now True
# then np.all returns True and we have a valid solution

#checks how many elements are missing from the state to the final solution (all True)
def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],     # state saves only the number of the taken subset
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
# creates a list of lists (subsets) that are in state.taken
# the reduce works in the same way of the previous method 
# so it returns an array of true and false 
# sum then treats Trues as 1s, so it counts the elements actually taken
# and the subtraction then returns how many elements are missing

def f(state):
    # the cost function is defined as g(n) being how many sets we have taken up to now
    # and h(n) being the estimated distance from now to the goal state
    return len(state.taken) + distance(state)


In [4]:
# assert checks if an expression returns True followed by a message in case of False
# the expression in this case is the goal_check function and the state that it
# takes as input is where all the sets are taken, verifying if the currents
# subsets permit the program to be solvable, otherwise the program will be stopped
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [1]:
frontier = PriorityQueue()  # set of all states that have been reached but not explored yet
                            # note that lower numbers equal higher priority
                            # in this case states with lower cost will be analyzed first
                            # (priority = current cost = numb of sets, content = state)
state = State(set(), set(range(NUM_SETS)))  # initialize state and sets taken as empty, while not_taken all subsets
frontier.put((f(state), state))  # adds the initial state to the frontier, using the f function previously defned
# note that h(n) depends on how many are missing, so we start from PROBLEM_SIZE as first key

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

NameError: name 'PriorityQueue' is not defined